# Step 1

### 5

In [39]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss

def SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, row, cho_matrix):
    dt = T / M  # T = maturity, M = number of time steps
    v = np.zeros((M + 1, Ite), dtype=np.float)
    v[0] = v0
    sdt = np.sqrt(dt)  # Sqrt of dt
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        v[t] = np.maximum(
            0,
            v[t - 1]
            + kappa * (theta - v[t - 1]) * dt
            + np.sqrt(v[t - 1]) * sigma * ran[row] * sdt,
        )
    return v

def Heston_paths(S0, r, v, row, cho_matrix):
    S = np.zeros((M + 1, Ite), dtype=float)
    S[0] = S0
    sdt = np.sqrt(dt)
    for t in range(1, M + 1, 1):
        ran = np.dot(cho_matrix, rand[:, t])
        S[t] = S[t - 1] * np.exp((r - 0.5 * v[t]) * dt + np.sqrt(v[t]) * ran[row] * sdt)

    return S

def random_number_gen(M, Ite):
    rand = np.random.standard_normal((2, M + 1, Ite))
    return rand

In [40]:
v0 = 0.032
kappa_v = 1.85
sigma_v = 0.35
theta_v = 0.045
rho = -0.3

S0 = 80  # Current underlying asset price
r = 0.055  # Risk-free rate
M0 = 252  # Number of time steps in a year (we decided to go for daily time steps)
T = 3/12  # Number of years
M = int(M0 * T)  # Total time steps
Ite = 100000  # Number of simulations
dt = T / M  # Length of time step

# Generating random numbers from standard normal
rand = random_number_gen(M, Ite)

# Covariance Matrix
covariance_matrix = np.zeros((2, 2), dtype=np.float)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

# Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S = Heston_paths(S0, r, V, 0, cho_matrix)

<ipython-input-40-8605e2cce7c6>:19: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  covariance_matrix = np.zeros((2, 2), dtype=np.float)
<ipython-input-39-34efa61912b0>:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  v = np.zeros((M + 1, Ite), dtype=np.float)


In [41]:
def heston_call_mc(S, K, r, T, t):
    payoff = np.maximum(0, S[-1, :] - K)

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

def heston_put_mc(S, K, r, T, t):
    payoff = np.maximum(0, K - S[-1, :])

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

In [42]:
np.random.seed(2)
print("European Call Price under Heston: ", round(heston_call_mc(S, 80, 0.055, 3/12, 0),2))

European Call Price under Heston:  2.86


In [43]:
np.random.seed(2)
print("European Put Price under Heston: ", round(heston_put_mc(S, 80, 0.055, 3/12, 0),2))

European Put Price under Heston:  2.82


### 6

In [44]:
v0 = 0.032
kappa_v = 1.85
sigma_v = 0.35
theta_v = 0.045
rho = -0.7

S0 = 80  # Current underlying asset price
r = 0.055  # Risk-free rate
M0 = 252  # Number of time steps in a year (we decided to go for daily time steps)
T = 3/12  # Number of years
M = int(M0 * T)  # Total time steps
Ite = 100000  # Number of simulations
dt = T / M  # Length of time step

# Generating random numbers from standard normal
rand = random_number_gen(M, Ite)

# Covariance Matrix
covariance_matrix = np.zeros((2, 2), dtype=np.float)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

# Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S = Heston_paths(S0, r, V, 0, cho_matrix)

<ipython-input-44-978618e41ee9>:19: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  covariance_matrix = np.zeros((2, 2), dtype=np.float)
<ipython-input-39-34efa61912b0>:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  v = np.zeros((M + 1, Ite), dtype=np.float)


In [45]:
np.random.seed(2)
print("European Call Price under Heston: ", round(heston_call_mc(S, 80, 0.055, 3/12, 0),2))

European Call Price under Heston:  2.11


In [46]:
np.random.seed(2)
print("European Put Price under Heston: ", round(heston_put_mc(S, 80, 0.055, 3/12, 0),2))

European Put Price under Heston:  3.44


### 7

DELTA

Correlation equal to -0.3

In [47]:
v0 = 0.032
kappa_v = 1.85
sigma_v = 0.35
theta_v = 0.045
rho = -0.3

S0 = 80  # Current underlying asset price
delta_S = 0.01 # 1% increase in current underlying asset price
r = 0.055  # Risk-free rate
M0 = 252  # Number of time steps in a year (we decided to go for daily time steps)
T = 3/12  # Number of years
M = int(M0 * T)  # Total time steps
Ite = 100000  # Number of simulations
dt = T / M  # Length of time step

# Generating random numbers from standard normal
rand = random_number_gen(M, Ite)

# Covariance Matrix
covariance_matrix = np.zeros((2, 2), dtype=np.float)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

# Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S = Heston_paths(S0 + delta_S, r, V, 0, cho_matrix)

<ipython-input-47-82ac8dd420f2>:20: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  covariance_matrix = np.zeros((2, 2), dtype=np.float)
<ipython-input-39-34efa61912b0>:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  v = np.zeros((M + 1, Ite), dtype=np.float)


In [48]:
def delta_hest_call_mc(S, K, r, T, t, delta_S):

    np.random.seed(2)
    original_price = heston_call_mc(S, K, r, T, t)
    new_price = heston_call_mc(S + delta_S, K, r, T, t)
    delta = (new_price - original_price) / delta_S
    return delta

In [49]:
print("Delta of the Call Option:", round(delta_hest_call_mc(S, 80, 0.055, 3/12, 0, 0.01),2))

Delta of the Call Option: 0.5


In [50]:
def delta_hest_put_mc(S, K, r, T, t, delta_S):

    np.random.seed(2)
    original_price = heston_put_mc(S, K, r, T, t)
    new_price = heston_put_mc(S + delta_S, K, r, T, t)
    delta = (new_price - original_price) / delta_S
    return delta

In [51]:
print("Delta of the Put Option:", round(delta_hest_put_mc(S, 80, 0.055, 3/12, 0, 0.01),2))

Delta of the Put Option: -0.48


Correlation equal to -0.7

In [52]:
v0 = 0.032
kappa_v = 1.85
sigma_v = 0.35
theta_v = 0.045
rho = -0.7

S0 = 80  # Current underlying asset price
r = 0.055  # Risk-free rate
M0 = 252  # Number of time steps in a year (we decided to go for daily time steps)
T = 3/12  # Number of years
M = int(M0 * T)  # Total time steps
Ite = 100000  # Number of simulations
dt = T / M  # Length of time step

# Generating random numbers from standard normal
rand = random_number_gen(M, Ite)

# Covariance Matrix
covariance_matrix = np.zeros((2, 2), dtype=np.float)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

# Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S = Heston_paths(S0, r, V, 0, cho_matrix)

<ipython-input-52-978618e41ee9>:19: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  covariance_matrix = np.zeros((2, 2), dtype=np.float)
<ipython-input-39-34efa61912b0>:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  v = np.zeros((M + 1, Ite), dtype=np.float)


In [53]:
np.random.seed(2)
print("Delta of the Call Option:", round(delta_hest_call_mc(S, 80, 0.055, 3/12, 0, 0.01),2))

Delta of the Call Option: 0.46


In [54]:
np.random.seed(2)
print("Delta of the Put Option:", round(delta_hest_put_mc(S, 80, 0.055, 3/12, 0, 0.01),2))

Delta of the Put Option: -0.53


GAMMA

Correlation equal to -0.3

In [55]:
v0 = 0.032
kappa_v = 1.85
sigma_v = 0.35
theta_v = 0.045
rho = -0.3

S0 = 80  # Current underlying asset price
delta_S = 0.01 # 1% increase in current underlying asset price
delta_S_2 = 0.02 # 2% additional increase in current underlying asset price
r = 0.055  # Risk-free rate
M0 = 252  # Number of time steps in a year (we decided to go for daily time steps)
T = 3/12  # Number of years
M = int(M0 * T)  # Total time steps
Ite = 100000  # Number of simulations
dt = T / M  # Length of time step

# Generating random numbers from standard normal
rand = random_number_gen(M, Ite)

# Covariance Matrix
covariance_matrix = np.zeros((2, 2), dtype=np.float)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

# Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S = Heston_paths(S0 + delta_S, r, V, 0, cho_matrix)

<ipython-input-55-1a75472f6876>:21: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  covariance_matrix = np.zeros((2, 2), dtype=np.float)
<ipython-input-39-34efa61912b0>:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  v = np.zeros((M + 1, Ite), dtype=np.float)


In [56]:
def gamma_hest_call_mc(S, K, r, T, t, delta_S):

    np.random.seed(2)
    delta1 = delta_hest_call_mc(S, K, r, T, t, delta_S)
    delta2 = delta_hest_call_mc(S + delta_S, K, r, T, t, delta_S)
    gamma = (delta2 - delta1) / delta_S
    return gamma

In [57]:
np.random.seed(2)
print("Gamma of the Call Option:", round(gamma_hest_call_mc(S, 80, 0.055, 3/12, 0, 0.01),2))

Gamma of the Call Option: 0.06


In [58]:
def gamma_hest_put_mc(S, K, r, T, t, delta_S):

    np.random.seed(2)
    delta1 = delta_hest_put_mc(S, K, r, T, t, delta_S)
    delta2 = delta_hest_put_mc(S + delta_S, K, r, T, t, delta_S)
    gamma = (delta2 - delta1) / delta_S
    return gamma

In [59]:
np.random.seed(2)
print("Gamma of the Put Option:", round(gamma_hest_put_mc(S, 80, 0.055, 3/12, 0, 0.01),2))

Gamma of the Put Option: 0.06


Correlation equal to -0.7

In [60]:
v0 = 0.032
kappa_v = 1.85
sigma_v = 0.35
theta_v = 0.045
rho = -0.7

S0 = 80  # Current underlying asset price
delta_S = 0.01 # 1% increase in current underlying asset price
delta_S_2 = 0.02 # 2% additional increase in current underlying asset price
r = 0.055  # Risk-free rate
M0 = 252  # Number of time steps in a year (we decided to go for daily time steps)
T = 3/12  # Number of years
M = int(M0 * T)  # Total time steps
Ite = 100000  # Number of simulations
dt = T / M  # Length of time step

# Generating random numbers from standard normal
rand = random_number_gen(M, Ite)

# Covariance Matrix
covariance_matrix = np.zeros((2, 2), dtype=np.float)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

# Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S = Heston_paths(S0 + delta_S, r, V, 0, cho_matrix)

<ipython-input-60-2195b80c38c0>:21: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  covariance_matrix = np.zeros((2, 2), dtype=np.float)
<ipython-input-39-34efa61912b0>:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  v = np.zeros((M + 1, Ite), dtype=np.float)


In [61]:
np.random.seed(2)
print("Gamma of the Call Option:", round(gamma_hest_call_mc(S, 80, 0.055, 3/12, 0, 0.01),2))

Gamma of the Call Option: 0.07


In [62]:
np.random.seed(2)
print("Gamma of the Put Option:", round(gamma_hest_put_mc(S, 80, 0.055, 3/12, 0, 0.01),2))

Gamma of the Put Option: 0.07


### 8

In [63]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss

np.random.seed(2)
lamb = 0.75  # Lambda of the model
mu = -0.5  # Mu
delta = 0.22  # Delta

r = 0.055  # Risk-free rate
sigma = 0.35  # Volatility
T = 3/12  # Maturity/time period (in years)
S0 = 80  # Current Stock Price

Ite = 100000  # Number of simulations (paths)
M0 = 252  # Number of time steps in a year (we decided to go for daily time steps)
M = int(M0 * T)  # Total time steps
dt = T / M  # Time-step

SM = np.zeros((M + 1, Ite))
SM[0] = S0

# rj
rj = lamb * (np.exp(mu + 0.5 * delta**2) - 1)

# Random numbers
z1 = np.random.standard_normal((M + 1, Ite))
z2 = np.random.standard_normal((M + 1, Ite))
y = np.random.poisson(lamb * dt, (M + 1, Ite))

for t in range(1, M + 1):
    SM[t] = SM[t - 1] * (
        np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
        + (np.exp(mu + delta * z2[t]) - 1) * y[t]
    )
    SM[t] = np.maximum(
        SM[t], 0.00001
    )  # To ensure that the price never goes below zero!

In [64]:
def merton_call_mc(S, K, r, T, t):
    payoff = np.maximum(0, S[-1, :] - K)

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

In [65]:
print("European Call Price under Merton: ", round(merton_call_mc(SM, 80, r, T, 0),2))

European Call Price under Merton:  8.26


In [66]:
def merton_put_mc(S, K, r, T, t):
    payoff = np.maximum(0, K - S[-1, :])

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

In [67]:
print("European Put Price under Merton: ", round(merton_put_mc(SM, 80, r, T, 0),2))

European Put Price under Merton:  7.19


### 9

In [68]:
np.random.seed(2)
lamb = 0.25  # Lambda of the model
mu = -0.5  # Mu
delta = 0.22  # Delta

r = 0.055  # Risk-free rate
sigma = 0.35  # Volatility
T = 3/12  # Maturity/time period (in years)
S0 = 80  # Current Stock Price

Ite = 100000  # Number of simulations (paths)
M0 = 252  # Number of time steps in a year (we decided to go for daily time steps)
M = int(M0 * T)  # Total time steps
dt = T / M  # Time-step

SM = np.zeros((M + 1, Ite))
SM[0] = S0

# rj
rj = lamb * (np.exp(mu + 0.5 * delta**2) - 1)

# Random numbers
z1 = np.random.standard_normal((M + 1, Ite))
z2 = np.random.standard_normal((M + 1, Ite))
y = np.random.poisson(lamb * dt, (M + 1, Ite))

for t in range(1, M + 1):
    SM[t] = SM[t - 1] * (
        np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
        + (np.exp(mu + delta * z2[t]) - 1) * y[t]
    )
    SM[t] = np.maximum(
        SM[t], 0.00001
    )  # To ensure that the price never goes below zero!

In [69]:
print("European Call Price under Merton: ", round(merton_call_mc(SM, 80, r, T, 0),2))

European Call Price under Merton:  6.79


In [70]:
print("European Put Price under Merton: ", round(merton_put_mc(SM, 80, r, T, 0),2))

European Put Price under Merton:  5.74


### 10

In [71]:
def delta_call_mc(S, K, r, T, t, delta_S):

    np.random.seed(2)
    original_price = merton_call_mc(S, K, r, T, t)
    new_price = merton_call_mc(S + delta_S, K, r, T, t)
    delta = (new_price - original_price) / delta_S
    return delta

def delta_put_mc(S, K, r, T, t, delta_S):

    np.random.seed(2)
    original_price = merton_put_mc(S, K, r, T, t)
    new_price = merton_put_mc(S + delta_S, K, r, T, t)
    delta = (new_price - original_price) / delta_S
    return delta

In [72]:
print("Delta of the Call Option:", round(delta_call_mc(SM, 80, 0.055, 3/12, 0, 0.01),2))
print("Delta of the Put Option:", round(delta_put_mc(SM, 80, 0.055, 3/12, 0, 0.01),2))

Delta of the Call Option: 0.51
Delta of the Put Option: -0.47


In [73]:
def gamma_call_mc(S, K, r, T, t, delta_S):

    np.random.seed(2)
    detla1 = delta_call_mc(S, K, r, T, t, delta_S)
    delta2 = delta_call_mc(S + delta_S, K, r, T, t, delta_S)
    gamma = (delta2 - detla1) / delta_S
    return gamma

def gamma_put_mc(S, K, r, T, t, delta_S):

    np.random.seed(2)
    detla1 = delta_put_mc(S, K, r, T, t, delta_S)
    delta2 = delta_put_mc(S + delta_S, K, r, T, t, delta_S)
    gamma = (delta2 - detla1) / delta_S
    return gamma

In [74]:
print("Gamma of the Call Option:", round(gamma_call_mc(SM, 80, 0.055, 3/12, 0, 0.01),2))
print("Gamma of the Put Option:", round(gamma_put_mc(SM, 80, 0.055, 3/12, 0, 0.01),2))

Gamma of the Call Option: 0.02
Gamma of the Put Option: 0.02


### 11

In [ ]:
# Put-Call parity under Heston model and Monte-Carlo simulation with rho = -0.3
round(2.88 + 80 * np.exp(-0.055 * (3/12)), 2) == round(80 + 2.82, 2)

False

In [ ]:
# Put-Call parity under Heston model and Monte-Carlo simulation with rho = -0.7
round(2.11 + 80 * np.exp(-0.055 * (3/12)), 2) == round(80 + 3.44, 2)

False

In [75]:
# Put-Call parity under Merton model with jump intensity parameter = 0.75
round(8.26 + 80 * np.exp(-0.055 * (3/12)), 2) == round(80 + 7.19, 2)

False

In [76]:
# Put-Call parity under Merton model with jump intensity parameter = 0.25
round(6.79 + 80 * np.exp(-0.055 * (3/12)), 2) == round(80 + 5.74, 2)

False

### 12

In [79]:
# Strike prices needed
moneyness = np.linspace(0.85, 1.15, 7)
strikes = np.zeros(7)
for i in range(len(moneyness)):
    strikes[i] = S0/moneyness[i]

print('List of strikes is', strikes)

List of strikes is [94.11764706 88.88888889 84.21052632 80.         76.19047619 72.72727273
 69.56521739]


HESTON

In [80]:
v0 = 0.032
kappa_v = 1.85
sigma_v = 0.35
theta_v = 0.045
rho = -0.3

S0 = 80  # Current underlying asset price
r = 0.055  # Risk-free rate
M0 = 252  # Number of time steps in a year (we decided to go for daily time steps)
T = 3/12  # Number of years
M = int(M0 * T)  # Total time steps
Ite = 100000  # Number of simulations
dt = T / M  # Length of time step

# Generating random numbers from standard normal
rand = random_number_gen(M, Ite)

# Covariance Matrix
covariance_matrix = np.zeros((2, 2), dtype=np.float)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

# Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S = Heston_paths(S0, r, V, 0, cho_matrix)

<ipython-input-80-8605e2cce7c6>:19: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  covariance_matrix = np.zeros((2, 2), dtype=np.float)
<ipython-input-39-34efa61912b0>:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  v = np.zeros((M + 1, Ite), dtype=np.float)


CALLS

In [81]:
np.random.seed(2)
print("European Call Price under Heston: ", round(heston_call_mc(S, strikes[0], 0.055, 3/12, 0),2))

European Call Price under Heston:  0.1


In [82]:
np.random.seed(2)
print("European Call Price under Heston: ", round(heston_call_mc(S, strikes[1], 0.055, 3/12, 0),2))

European Call Price under Heston:  0.4


In [83]:
np.random.seed(2)
print("European Call Price under Heston: ", round(heston_call_mc(S, strikes[2], 0.055, 3/12, 0),2))

European Call Price under Heston:  1.23


In [84]:
np.random.seed(2)
print("European Call Price under Heston: ", round(heston_call_mc(S, strikes[3], 0.055, 3/12, 0),2))

European Call Price under Heston:  2.86


In [85]:
np.random.seed(2)
print("European Call Price under Heston: ", round(heston_call_mc(S, strikes[4], 0.055, 3/12, 0),2))

European Call Price under Heston:  5.17


In [86]:
np.random.seed(2)
print("European Call Price under Heston: ", round(heston_call_mc(S, strikes[5], 0.055, 3/12, 0),2))

European Call Price under Heston:  7.85


In [87]:
np.random.seed(2)
print("European Call Price under Heston: ", round(heston_call_mc(S, strikes[6], 0.055, 3/12, 0),2))

European Call Price under Heston:  10.62


PUTS

In [88]:
np.random.seed(2)
print("European Put Price under Heston: ", round(heston_put_mc(S, strikes[0], 0.055, 3/12, 0),2))

European Put Price under Heston:  13.99


In [89]:
np.random.seed(2)
print("European Put Price under Heston: ", round(heston_put_mc(S, strikes[1], 0.055, 3/12, 0),2))

European Put Price under Heston:  9.13


In [90]:
np.random.seed(2)
print("European Put Price under Heston: ", round(heston_put_mc(S, strikes[2], 0.055, 3/12, 0),2))

European Put Price under Heston:  5.35


In [91]:
np.random.seed(2)
print("European Put Price under Heston: ", round(heston_put_mc(S, strikes[3], 0.055, 3/12, 0),2))

European Put Price under Heston:  2.82


In [92]:
np.random.seed(2)
print("European Put Price under Heston: ", round(heston_put_mc(S, strikes[4], 0.055, 3/12, 0),2))

European Put Price under Heston:  1.37


In [93]:
np.random.seed(2)
print("European Put Price under Heston: ", round(heston_put_mc(S, strikes[5], 0.055, 3/12, 0),2))

European Put Price under Heston:  0.64


In [94]:
np.random.seed(2)
print("European Put Price under Heston: ", round(heston_put_mc(S, strikes[6], 0.055, 3/12, 0),2))

European Put Price under Heston:  0.29


MERTON

In [95]:
np.random.seed(2)
lamb = 0.75  # Lambda of the model
mu = -0.5  # Mu
delta = 0.22  # Delta

r = 0.055  # Risk-free rate
sigma = 0.35  # Volatility
T = 3/12  # Maturity/time period (in years)
S0 = 80  # Current Stock Price

Ite = 100000  # Number of simulations (paths)
M0 = 252  # Number of time steps in a year (we decided to go for daily time steps)
M = int(M0 * T)  # Total time steps
dt = T / M  # Time-step

SM = np.zeros((M + 1, Ite))
SM[0] = S0

# rj
rj = lamb * (np.exp(mu + 0.5 * delta**2) - 1)

# Random numbers
z1 = np.random.standard_normal((M + 1, Ite))
z2 = np.random.standard_normal((M + 1, Ite))
y = np.random.poisson(lamb * dt, (M + 1, Ite))

for t in range(1, M + 1):
    SM[t] = SM[t - 1] * (
        np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
        + (np.exp(mu + delta * z2[t]) - 1) * y[t]
    )
    SM[t] = np.maximum(
        SM[t], 0.00001
    )  # To ensure that the price never goes below zero!

CALLS

In [96]:
print("European Call Price under Merton: ", round(merton_call_mc(SM, strikes[0], r, T, 0),2))

European Call Price under Merton:  2.77


In [97]:
print("European Call Price under Merton: ", round(merton_call_mc(SM, strikes[1], r, T, 0),2))

European Call Price under Merton:  4.31


In [98]:
print("European Call Price under Merton: ", round(merton_call_mc(SM, strikes[2], r, T, 0),2))

European Call Price under Merton:  6.17


In [99]:
print("European Call Price under Merton: ", round(merton_call_mc(SM, strikes[3], r, T, 0),2))

European Call Price under Merton:  8.26


In [115]:
print("European Call Price under Merton: ", round(merton_call_mc(SM, strikes[4], r, T, 0),2))

European Call Price under Merton:  10.5


In [100]:
print("European Call Price under Merton: ", round(merton_call_mc(SM, strikes[5], r, T, 0),2))

European Call Price under Merton:  12.79


In [101]:
print("European Call Price under Merton: ", round(merton_call_mc(SM, strikes[6], r, T, 0),2))

European Call Price under Merton:  15.08


PUTS

In [102]:
print("European Put Price under Merton: ", round(merton_put_mc(SM, strikes[0], r, T, 0),2))

European Put Price under Merton:  15.62


In [103]:
print("European Put Price under Merton: ", round(merton_put_mc(SM, strikes[1], r, T, 0),2))

European Put Price under Merton:  12.0


In [104]:
print("European Put Price under Merton: ", round(merton_put_mc(SM, strikes[2], r, T, 0),2))

European Put Price under Merton:  9.25


In [105]:
print("European Put Price under Merton: ", round(merton_put_mc(SM, strikes[3], r, T, 0),2))

European Put Price under Merton:  7.19


In [106]:
print("European Put Price under Merton: ", round(merton_put_mc(SM, strikes[4], r, T, 0),2))

European Put Price under Merton:  5.67


In [107]:
print("European Put Price under Merton: ", round(merton_put_mc(SM, strikes[5], r, T, 0),2))

European Put Price under Merton:  4.55


In [108]:
print("European Put Price under Merton: ", round(merton_put_mc(SM, strikes[6], r, T, 0),2))

European Put Price under Merton:  3.71


# Step 2

### 13

In [112]:
def SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, row, cho_matrix):
    dt = T / M  # T = maturity, M = number of time steps
    v = np.zeros((M + 1, Ite), dtype=np.float)
    v[0] = v0
    sdt = np.sqrt(dt)  # Sqrt of dt
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        v[t] = np.maximum(
            0,
            v[t - 1]
            + kappa * (theta - v[t - 1]) * dt
            + np.sqrt(v[t - 1]) * sigma * ran[row] * sdt,
        )
    return v

def Heston_paths(S0, r, v, row, cho_matrix):
    S = np.zeros((M + 1, Ite), dtype=float)
    S[0] = S0
    sdt = np.sqrt(dt)
    for t in range(1, M + 1, 1):
        ran = np.dot(cho_matrix, rand[:, t])
        S[t] = S[t - 1] * np.exp((r - 0.5 * v[t]) * dt + np.sqrt(v[t]) * ran[row] * sdt)

    return S

def random_number_gen(M, Ite):
    rand = np.random.standard_normal((2, M + 1, Ite))
    return rand
np.random.seed(2)
v0 = 0.032
kappa_v = 1.85
sigma_v = 0.35
theta_v = 0.045
rho = -0.3

S0 = 80  # Current underlying asset price
r = 0.055  # Risk-free rate
M0 = 252  # Number of time steps in a year (we decided to go for daily time steps)
T = 3/12  # Number of years
M = int(M0 * T)  # Total time steps
Ite = 100000  # Number of simulations
dt = T / M  # Length of time step

# Generating random numbers from standard normal
rand = random_number_gen(M, Ite)


# Covariance Matrix
covariance_matrix = np.zeros((2, 2), dtype=np.float)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

# Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S = Heston_paths(S0, r, V, 0, cho_matrix)

def heston_call_american(S, K, r, T, t, n_steps):

    payoff = np.maximum(S.T - K, 0)

    option_value = payoff[:, -1]

    for i in range(n_steps-1, -1, -1):
        discount_factor = np.exp(-r * dt)
        option_value = np.maximum(payoff[:, i], option_value * discount_factor)

    average = np.mean(option_value)


    return average

print("American Call Price under Heston: ", round(heston_call_american(S, 80, r, T, 0, M),2))

<ipython-input-112-d4a58b1a5dd5>:49: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  covariance_matrix = np.zeros((2, 2), dtype=np.float)
<ipython-input-112-d4a58b1a5dd5>:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  v = np.zeros((M + 1, Ite), dtype=np.float)


American Call Price under Heston:  5.15


The European Call price found under Heston in Question 5 was 2.86, while under the same model the American Call price stands at 5.15. This makes sense, given that the American call price should be higher than or equal to the European call price but never lower, due to the higher cost to be paid for the early exercise feature.

In [113]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss
np.random.seed(2)
lamb = 0.75  # Lambda of the model
mu = -0.5  # Mu
delta = 0.22  # Delta

r = 0.055  # Risk-free rate
sigma = 0.35  # Volatility
T = 3/12  # Maturity/time period (in years)
S0 = 80  # Current Stock Price

Ite = 100000  # Number of simulations (paths)
M0 = 252  # Number of time steps in a year (we decided to go for daily time steps)
M = int(M0 * T)  # Total time steps
dt = T / M  # Time-step

SM = np.zeros((M + 1, Ite))
SM[0] = S0

# rj
rj = lamb * (np.exp(mu + 0.5 * delta**2) - 1)

# Random numbers
z1 = np.random.standard_normal((M + 1, Ite))
z2 = np.random.standard_normal((M + 1, Ite))
y = np.random.poisson(lamb * dt, (M + 1, Ite))

for t in range(1, M + 1):
    SM[t] = SM[t - 1] * (
        np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
        + (np.exp(mu + delta * z2[t]) - 1) * y[t]
    )
    SM[t] = np.maximum(
        SM[t], 0.00001
    )  # To ensure that the price never goes below zero!

def merton_call_american(S, K, r, T, t, n_steps):

    payoff = np.maximum(S.T - K, 0)

    option_value = payoff[:, -1]

    for i in range(n_steps-1, -1, -1):
        discount_factor = np.exp(-r * dt)
        option_value = np.maximum(payoff[:, i], option_value * discount_factor)

    average = np.mean(option_value)


    return average

print("American Call Price under Merton: ", round(merton_call_american(SM, 80, r, T, 0, M),2))

American Call Price under Merton:  13.88


The European Call price found under Merton in Question 8 was 8.26, while under the same model the American Call price stands at 13.88. This makes sense, given that the American call price should be higher than or equal to the European call price but never lower, due to the higher cost to be paid for the early exercise feature.

### 14

In [ ]:
v0 = 0.032
kappa_v = 1.85
sigma_v = 0.35
theta_v = 0.045
rho = -0.7

S0 = 80  # Current underlying asset price
r = 0.055  # Risk-free rate
M0 = 252  # Number of time steps in a year (we decided to go for daily time steps)
T = 3/12  # Number of years
M = int(M0 * T)  # Total time steps
Ite = 100000  # Number of simulations
dt = T / M  # Length of time step

# Generating random numbers from standard normal
rand = random_number_gen(M, Ite)

# Covariance Matrix
covariance_matrix = np.zeros((2, 2), dtype=np.float)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

# Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S = Heston_paths(S0, r, V, 0, cho_matrix)

def heston_call_mc_with_barrier(S, K, r, T, t, barrier_level):
    # Check if the barrier is crossed during the simulation
    barrier_crossed = np.any(S > barrier_level, axis=0)

    # Calculate the payoff based on barrier crossing
    payoff = np.where(barrier_crossed, np.maximum(0, S[-1, :] - K), 0)

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average


np.random.seed(2)
print("UAI European Call Price under Heston: ", round(heston_call_mc_with_barrier(S, 95, 0.055, 3/12, 0, 95),2))



<ipython-input-43-484ddf8710fa>:19: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  covariance_matrix = np.zeros((2, 2), dtype=np.float)
<ipython-input-33-62c3d9e7d8d3>:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  v = np.zeros((M + 1, Ite), dtype=np.float)


UAI European Call Price under Heston:  0.01


The European Call price found under Heston in Question 6 was 2.11, while under the same model data the UAI European Call price stands at 0.01. This makes sense, given that the UAI European Call option becomes alive only if the stock price reaches the barrier level, so it has less probability to pay off, leaving aside the fact that becomes active only under certain conditions. On the other hand, the simple european call option is always alive, thus gives us more possibilities to pay off. That is why the European call price should be higher than UAI European call price and this is exactly the case.

### 15

In [114]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss

np.random.seed(2)
lamb = 0.75  # Lambda of the model
mu = -0.5  # Mu
delta = 0.22  # Delta

r = 0.055  # Risk-free rate
sigma = 0.35  # Volatility
T = 3/12  # Maturity/time period (in years)
S0 = 80  # Current Stock Price

Ite = 100000  # Number of simulations (paths)
M0 = 252  # Number of time steps in a year (we decided to go for daily time steps)
M = int(M0 * T)  # Total time steps
dt = T / M  # Time-step

SM = np.zeros((M + 1, Ite))
SM[0] = S0

# rj
rj = lamb * (np.exp(mu + 0.5 * delta**2) - 1)

# Random numbers
z1 = np.random.standard_normal((M + 1, Ite))
z2 = np.random.standard_normal((M + 1, Ite))
y = np.random.poisson(lamb * dt, (M + 1, Ite))

for t in range(1, M + 1):
    SM[t] = SM[t - 1] * (
        np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
        + (np.exp(mu + delta * z2[t]) - 1) * y[t]
    )
    SM[t] = np.maximum(
        SM[t], 0.00001
    )  # To ensure that the price never goes below zero!


def merton_put_mc_with_barrier(S, K, r, T, t, barrier_level):
    # Check if the barrier is crossed during the simulation
    barrier_crossed = np.any(S < barrier_level, axis=0)

    # Calculate the payoff based on barrier crossing
    payoff = np.where(barrier_crossed, np.maximum(0, K - S[-1, :]), 0)

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average


print("DAI European Call Price under Merton: ", round(merton_put_mc_with_barrier(SM, 65, r, T, 0, 65),2))

DAI European Call Price under Merton:  2.76


The European Call price found under Merton in Question 8 was 8.26, while under the same model data the DAI European Call price stands at 2.76. This makes sense, for the same reasons explained in point 14.